In [1]:
import pathlib, warnings
warnings.filterwarnings('ignore')

import pandas as pd
import plotly.express as px
print(pd.__version__)


DATA_DIR = pathlib.Path('data')

2.2.1


# Health Data

In [2]:
health = (pd
    .read_csv(DATA_DIR / 'healthcare_dataset.csv')
    .rename(columns=lambda x: x.lower().replace(' ', '_'))
)

In [3]:
health.head()

,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal


In [4]:
health.columns

Index(['name', 'age', 'gender', 'blood_type', 'medical_condition',
       'date_of_admission', 'doctor', 'hospital', 'insurance_provider',
       'billing_amount', 'room_number', 'admission_type', 'discharge_date',
       'medication', 'test_results'],
      dtype='object')

In [5]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                10000 non-null  object 
 1   age                 10000 non-null  int64  
 2   gender              10000 non-null  object 
 3   blood_type          10000 non-null  object 
 4   medical_condition   10000 non-null  object 
 5   date_of_admission   10000 non-null  object 
 6   doctor              10000 non-null  object 
 7   hospital            10000 non-null  object 
 8   insurance_provider  10000 non-null  object 
 9   billing_amount      10000 non-null  float64
 10  room_number         10000 non-null  int64  
 11  admission_type      10000 non-null  object 
 12  discharge_date      10000 non-null  object 
 13  medication          10000 non-null  object 
 14  test_results        10000 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 1.1+

In [6]:
DTTYPE = 'datetime64[ns]'
CATEGORICAL_COLUMNS = [
    'gender', 'blood_type', 'medical_condition',
    'hospital', 'insurance_provider', 'admission_type',
    'medication', 'test_results'
]
health = (health
    .assign(time_in_hospital=health.discharge_date.astype(DTTYPE) - health.date_of_admission.astype(DTTYPE))
    .drop(columns=['date_of_admission', 'discharge_date', 'room_number'])
    .astype({col: 'category' for col in CATEGORICAL_COLUMNS})
)

In [7]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   name                10000 non-null  object         
 1   age                 10000 non-null  int64          
 2   gender              10000 non-null  category       
 3   blood_type          10000 non-null  category       
 4   medical_condition   10000 non-null  category       
 5   doctor              10000 non-null  object         
 6   hospital            10000 non-null  category       
 7   insurance_provider  10000 non-null  category       
 8   billing_amount      10000 non-null  float64        
 9   admission_type      10000 non-null  category       
 10  medication          10000 non-null  category       
 11  test_results        10000 non-null  category       
 12  time_in_hospital    10000 non-null  timedelta64[ns]
dtypes: category(8), float64(1), int6

## Analysis

In [8]:
health.groupby('blood_type')['test_results'].value_counts()

blood_type  test_results
A+          Normal          427
            Abnormal        411
            Inconclusive    403
A-          Abnormal        437
            Normal          410
            Inconclusive    391
AB+         Abnormal        432
            Inconclusive    427
            Normal          399
AB-         Abnormal        449
            Inconclusive    425
            Normal          401
B+          Inconclusive    424
            Abnormal        422
            Normal          398
B-          Abnormal        456
            Normal          406
            Inconclusive    390
O+          Inconclusive    427
            Abnormal        417
            Normal          404
O-          Abnormal        432
            Normal          422
            Inconclusive    390
Name: count, dtype: int64

In [9]:
test_result_by_blood_type_bars = px.histogram(
    health,
    x='blood_type',
    color='test_results',
    barmode='group',
    histfunc='count',
    title='Blood Type vs Test Results',
    labels={'test_results': 'Test Results', 'blood_type': 'Blood Type'},
    template='plotly_dark'
)
test_result_by_blood_type_bars.update_layout(legend=dict(orientation='h',yanchor='top',y=1,xanchor='right',x=1))
test_result_by_blood_type_bars.show()

In [10]:
health.groupby('admission_type')['billing_amount'].mean()

admission_type
Elective     25891.832668
Emergency    24708.511933
Urgent       25960.833557
Name: billing_amount, dtype: float64

In [11]:
health.pivot_table(
    columns='blood_type',
    index='gender',
    values='billing_amount',
    aggfunc=['count', 'mean']
)

count                                            mean  \
blood_type    A+   A-  AB+  AB-   B+   B-   O+   O-           A+   
gender                                                             
Female       629  633  637  671  640  635  641  589  23975.35364   
Male         612  605  621  604  604  617  607  655  25656.10797   

                                                                    \
blood_type            A-           AB+           AB-            B+   
gender                                                               
Female      25949.510503  26644.341226  25756.117601  25286.571130   
Male        26557.170205  25163.018441  24954.755695  25760.796633   

                                                      
blood_type            B-            O+            O-  
gender                                                
Female      25688.592743  24781.000084  25792.246994  
Male        25666.542745  25308.203236  25357.897898

In [31]:
mean_billing_amount_by_age = px.scatter(
    (
        health
        .groupby('age')
        ['billing_amount']
        .aggregate(['count', 'mean'])
        .reset_index()
    ),
    x='age',
    y='mean',
    color='count',
    template='plotly_dark',
)
mean_billing_amount_by_age.update_traces(mode='lines+markers', line_dash='dash')
mean_billing_amount_by_age.show()